In [3]:
import pandas as pd 
import numpy as np 
import pyodbc
import ast
from datetime import datetime
import json
import re

import sys
import os

In [6]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                              'Server=UTIL-PROD-DB;'
                              'Database=NewClientInfo;')

def string_to_list(s):
    try:
        result = ast.literal_eval(s)
        if isinstance(result, list):
            return result
        else:
            raise ValueError("The string does not represent a list.")
    except Exception as e:
        print(f"Error: {e}")
        return None
    
def GetChargeHistory(conn1):
    sql1 = """SELECT TOP (1000) [Id]
        ,[RateAcuityRateId]
        ,[EffectiveDate]
        ,[RateHistory]
        ,[CreatedBy]
        ,[CreatedDate]
        ,[ModifiedBy]
        ,[ModifiedDate]
    FROM [ExternalData].[dbo].[RateAcuityRateHistory]
    WHERE RateAcuityRateId = 22
    """
    return string_to_list(pd.read_sql(sql1,conn1)['RateHistory'][0])[0]

def GetChargeConfig(conn1):
    sql1 = """
    SELECT TOP (1000) [Id]
        ,[RateAcuityRateId]
        ,[RateAcuityChargeId]
        ,[RateAcuityChargeDescription] AS 'Description'
        ,[RateAcuityChargeSeason]
        ,[RateAcuityChargeStartDate]
        ,[RateAcuityChargeEndDate]
        ,[RateAcuityChargeStartTime]
        ,[RateAcuityChargeEndTime]
        ,[RateAcuityChargeTimeOfDay]
        ,[RateAcuityChargeDeterminant] AS 'Unit'
        ,[ChargeTypeId]
        ,[ChargeParameterTypeId]
        ,[UsageType]
        ,[WeightedAverage] AS 'Weighted'
        ,[Prorated]
        ,[Block]
        ,[ServiceTypeId]
        ,[RatchetChargeDetailsId]
        ,[Complete]
        ,[CreatedBy]
        ,[CreatedDate]
        ,[ModifiedBy]
        ,[ModifiedDate]
    FROM [ExternalData].[dbo].[ChargeConfiguration]
    WHERE RateAcuityRateId = 1
    AND RateAcuityChargeDescription NOT LIKE '%Average Supply Charge%'
    AND [ChargeTypeId] IN (0, 2)

    """
    return pd.read_sql(sql1,conn1)

In [8]:
charge_history = GetChargeHistory(conn1)
charge_config = GetChargeConfig(conn1)

C:\Users\local_nmilinkovich\Temp\ipykernel_33536\3308119877.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return string_to_list(pd.read_sql(sql1,conn1)['RateHistory'][0])[0]
C:\Users\local_nmilinkovich\Temp\ipykernel_33536\3308119877.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql1,conn1)


In [15]:
charge_config.to_csv(r'C:\Users\nmilinkovich\Documents\ideal-tele\charge_config.csv')
with open(r'C:\Users\nmilinkovich\Documents\ideal-tele\charge_history.json', 'w') as f:
    json.dump(charge_history, f)